In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [6]:
df = pd.read_csv('../data/train_data_encoded.csv')
print('shape:', df.shape)
df.head(5)

shape: (28942, 7)


,weight,quality,color,clarity,table_percent,price,depth_percent
0,0.30,3,6,1,0.58,6.353,0.624
1,1.01,4,5,5,0.56,9.183,0.627
2,0.72,4,4,3,0.59,7.983,0.618
3,1.08,2,3,1,0.57,8.371,0.632
4,0.36,3,3,4,0.59,6.588,0.623


In [7]:
#split dara train and test
X = df.drop(["price"], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())
print('\n')
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

Datos de entrenamiento
-----------------------
count    23153.000000
mean         7.992375
std          0.993806
min          5.787000
25%          7.156000
50%          8.082000
75%          8.755000
max          9.842000
Name: price, dtype: float64


Datos de testeo
-----------------------
count    5789.000000
mean        8.006556
std         0.986775
min         5.866000
25%         7.189000
50%         8.089000
75%         8.755000
max         9.841000
Name: price, dtype: float64


In [8]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [9]:
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=8, random_state=0, criterion='squared_error')
gb.fit(X_train, y_train)

y_pred_gb_test= gb.predict(X_test)
y_pred_gb_train= gb.predict(X_train)

results_gradient_3 = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "Gradient 3")
results_gradient_3.style.background_gradient(cmap='coolwarm')

,MAE,MSE,RMSE,R2,set,modelo
0,0.074649,0.009997,0.099986,0.989731,test,Gradient 3
1,0.058890,0.006233,0.078951,0.993688,train,Gradient 3


In [13]:
df_test = pd.read_csv('../data/test_data_encoded.csv')
df_test

,weight,quality,color,clarity,table_percent,depth_percent
0,0.32,4,1,2,0.580,0.605
1,1.24,3,1,2,0.600,0.629
2,1.66,3,6,2,0.590,0.620
3,0.75,3,6,1,0.560,0.606
4,1.50,0,5,1,0.550,0.648
...,...,...,...,...,...,...
13480,1.10,3,3,2,0.600,0.596
13481,0.90,2,6,2,0.600,0.621
13482,0.30,4,4,3,0.533,0.621
13483,1.25,4,1,2,0.590,0.596


In [28]:
#predict test to submit competition
y_pred_gb_test = gb.predict(df_test)
submission_1 = pd.DataFrame(y_pred_gb_test).reset_index()
print('shape:', y_pred_gb_test.shape)
submission_1.columns = ['id', 'price']
submission_1

shape: (13485,)


,id,price
0,0,6.131006
1,1,8.602525
2,2,9.466583
3,3,7.872835
4,4,9.067332
...,...,...
13480,13480,8.559315
13481,13481,8.380205
13482,13482,6.496367
13483,13483,8.653308


In [29]:
submission_1.to_csv('../output/submission_1_fs.csv', index=False)